<a href="https://colab.research.google.com/github/EleonoraBartolomucci/Fairness/blob/master/Faceplusplus/ElaboraFoto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install clarifai

     |████████████████████████████████| 133kB 4.9MB/s 
  Created wheel for clarifai: filename=clarifai-2.6.2-cp36-none-any.whl size=188403 sha256=d11667b7de000ba390f64700dda16e6e58755bc73019f0613e1a217a1620e20c
  Stored in directory: /root/.cache/pip/wheels/17/2c/3b/d89ffd4572633f70872165cedf61813ac8047461ed5e1e3768
Successfully built clarifai


In [0]:
import json
import csv

# apro e parso il file .json
def convert(response):

  with open('utenti.txt', 'w') as outfile:
    json.dump(response, outfile) 
        
  # apro e parso il file .json
  data = open('utenti.txt').read()
  data_parsed = json.loads(data)
  
  #creo e apro un file .csv per scrivere
  file_parsed = open('File_Parsered.csv', 'w')
    
  #creo il writer
  writer = csv.writer(file_parsed)

  #creo header del csv
  header = ['id', 'age', 'gender', 'ethnicity']
  writer.writerow(header)

  #ciclo per estrarre dal file json i valori di id, età, genere ed etnia
  for person in data_parsed['user']:
     writer.writerow([person['id'],
                      person['age'],
                      person['gender'],
                      person['ethnicity']])
     
  
  #salvo e chiudo il file .csv
  file_parsed.close()

In [0]:
from clarifai.rest import ClarifaiApp
import time

def detect_clarifai(id, url_photo, data):
  app = ClarifaiApp(api_key='6651c2b2e8d64d3291238ef15277c18c')
  model = app.models.get(model_id="c0c0ac362b03416da06ab3fa36fb58e3")
  response = model.predict_by_url(url = '%s' % url_photo)

  if response['outputs'][0]['data'] != {}:
    x = 0
    while (x < len(response['outputs'][0]['data']['regions'])):
      age = response['outputs'][0]['data']['regions'][x]['data']['face']['age_appearance']['concepts'][0]['name']
      gender = response['outputs'][0]['data']['regions'][x]['data']['face']['gender_appearance']['concepts'][0]['name']
      ethnicity = response['outputs'][0]['data']['regions'][x]['data']['face']['multicultural_appearance']['concepts'][0]['name']

      data['user'].append({
         'id' : '%s' % id,
         'age' : '%s' % age,
         'gender' : '%s' % gender,
         'ethnicity' : '%s' % ethnicity,
      })
      x = x + 1
  else:
    return

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
from google.colab import auth
from clarifai.rest import ApiError
import json

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Auto-iterate through all files that matches this query
file_list = drive.ListFile({'q': "'18gjR0tXLt6ET54X-EcUZFBkS2u8Sn1R1' in parents and trashed=false"}).GetList()

data = {}
data['user'] = []
for file1 in file_list:
  id = file1['title'] 
  detect_clarifai(id.rstrip('.jpg'), file1['thumbnailLink'], data)

convert(data)